# HUFS-OOS-cs-student-ragbot AWS EC2 배포 및 DuckDNS 연결 가이드

이 문서는 `HUFS-OOS-cs-student-ragbot` 프로젝트를 AWS EC2에 배포하고, DuckDNS를 통해 외부에서 접속 가능한 웹서비스로 만드는 전 과정을 정리한 가이드입니다.

---

## ✅ 1. EC2 인스턴스 생성 및 기본 설정

1. AWS EC2 대시보드 접속 → Ubuntu 22.04 LTS 인스턴스 생성
2. 인스턴스 유형: `t2.micro` 이상
3. 보안 그룹 설정:  
   - **SSH (22)**  
   - **HTTP (80)**  
   - **HTTPS (443)**
4. `.pem` 키파일 다운로드 (예: `HUFSchatbot.pem`)

### SSH 접속

```bash
chmod 400 HUFSchatbot.pem
ssh -i "HUFSchatbot.pem" ubuntu@43.201.157.231
```

---

## ✅ 2. 서버 환경 설정

```bash
sudo apt update && sudo apt upgrade -y
sudo apt install git python3-pip python3-venv nginx -y
```

---

## ✅ 3. 프로젝트 클론 및 Python 가상환경 설정

```bash
git clone https://github.com/leegeonu02/HUFS-OOS-cs-student-ragbot.git
cd HUFS-OOS-cs-student-ragbot

python3 -m venv venv
source venv/bin/activate
pip install --upgrade pip
pip install -r requirements.txt
```

---

## ✅ 4. FastAPI 서버 실행 테스트

```bash
uvicorn main:app --host 0.0.0.0 --port 8000
```

- 접속: http://43.201.157.231:8000/docs

---

## ✅ 5. DuckDNS 도메인 설정

1.	https://www.duckdns.org 에서 로그인 후 서브도메인 생성 (예: hufscomchatbot)
2.	DuckDNS 갱신 스크립트 작성

```bash
mkdir ~/duckdns
cd ~/duckdns
nano duck.sh
```

```bash
# duck.sh 내용
echo "url=https://www.duckdns.org/update?domains=hufscomchatbot&token=<YOUR_TOKEN>&ip=" | curl -k -o ~/duckdns/duck.log -K -
```

```bash
chmod 700 duck.sh
crontab -e
```

```bash
*/5 * * * * ~/duckdns/duck.sh >/dev/null 2>&1
```

---

## ✅ 6. Nginx 리버스 프록시 설정

```bash
sudo nano /etc/nginx/sites-available/ragbot
```

```bash
server {
    listen 80;
    server_name hufscomchatbot.duckdns.org;

    location / {
        proxy_pass http://127.0.0.1:8000;
        proxy_set_header Host $host;
        proxy_set_header X-Real-IP $remote_addr;
    }
}
```

```bash
sudo ln -s /etc/nginx/sites-available/ragbot /etc/nginx/sites-enabled/
sudo nginx -t
sudo systemctl restart nginx
```

---

## ✅ 7. FastAPI 백그라운드 실행 (선택)

```bash
nano launch.sh
```

```bash
#!/bin/bash
source ~/HUFS-OOS-cs-student-ragbot/venv/bin/activate
cd ~/HUFS-OOS-cs-student-ragbot
uvicorn main:app --host 0.0.0.0 --port 8000
```

```bash
chmod +x launch.sh
nohup ./launch.sh > app.log 2>&1 &
```

---

## ✅ 8. 클라이언트 웹 코드 배포 (옵션)

```bash
cd client_web
python3 -m http.server 3000
```

또는 Nginx 정적 파일 호스팅으로 설정

---

## ✅ 9. 접속 확인

	•	접속 주소:
🔗 http://hufscomchatbot.duckdns.org

---

### 오류 해결

1. 디스크 공간 부족

```bash
df -h
sudo apt clean
sudo rm -rf ~/.cache/pip
```

2. SSH 연결 오류

- .pem파일 권한 오류 시:

```bash
chmod 400 HUFSchatbot.pem
```

- Permission denied (publickey) 오류 시 사용자명이 ubuntu인지 확인